In [1]:
import pandas as pd

In [19]:
df = pd.read_csv('dataset/train_data.csv')

#df = df.dropna(how='any')
#df = df.reset_index(drop=True)

df.head()

,UID,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,od_start_time,...,cutoff_timestamp,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,actual_time
0,lWYYrbNEdf,2018-09-15 02:23:18.656742,thanos::sroute:9d8ed3f7-a8af-4393-ad7a-c57a720...,FTL,trip-153697819865649878,IND832109AAB,Jamshedpur_Central_I_3 (Jharkhand),IND834002AAB,Ranchi_Hub (Jharkhand),2018-09-15 02:23:18.656742,...,2018-09-15 04:45:23,158.807515,778.672716,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,100.0
1,iNJcHQlPCm,2018-09-30 15:07:48.971326,thanos::sroute:b9142a73-d68f-46ec-9afc-d145479...,FTL,trip-153832006897106333,IND422011AAD,Nashik_TgrniaRD_I (Maharashtra),IND421302AAG,Bhiwandi_Mankoli_HB (Maharashtra),2018-09-30 22:38:13.903069,...,2018-10-01 05:38:30,81.388109,227.479029,NaN,3.625248,68.517914,18.787231,34.612259,4.277350,61.0
2,U6N0M43qwR,2018-09-23 22:15:46.635975,thanos::sroute:883e99fa-50a3-40e0-a2e2-9b12ed6...,FTL,trip-153774094663572416,IND845305AAA,Raxaul_KairiyaT_D (Bihar),NaN,Muzaffrpur_Bbganj_I (Bihar),2018-09-24 03:56:37.016232,...,2018-09-24 09:21:30,134.486898,400.072069,1188.943158,4.415969,70.475386,24.371978,29.867950,5.322072,69.0
3,RrJUneuYNj,2018-09-21 05:11:44.833176,thanos::sroute:96a80600-40e1-436b-9161-fa68f9e...,FTL,trip-153750670483274503,IND842001AAA,Muzaffrpur_Bbganj_I (Bihar),IND854326AAB,Purnia_Central_H_2 (Bihar),2018-09-22 10:08:32.593022,...,2018-09-22 14:09:17,262.866674,510.123878,999.421258,2.946647,101.876844,19.588977,27.794019,4.265799,300.0
4,7QLdD7iRC9,2018-09-18 01:10:49.936747,thanos::sroute:b43ec86d-59c1-452c-b4a9-807d357...,FTL,trip-153723305402762987,IND411033AAA,Pune_Tathawde_H (Maharashtra),IND501359AAE,Hyderabad_Shamshbd_H (Telangana),2018-09-18 01:10:49.936747,...,2018-09-18 12:14:05,379.389280,NaN,NaN,1.998790,46.571301,22.671495,26.971844,3.553652,286.0


In [3]:
df['segment_key'] = df['trip_uuid'] + df['source_name'] + df['destination_name']
print(len(pd.unique(df['segment_key'])))

22126


In [21]:
df.columns

Index(['UID', 'trip_creation_time', 'route_schedule_uuid', 'route_type',
       'trip_uuid', 'source_center', 'source_name', 'destination_center',
       'destination_name', 'od_start_time', 'od_end_time',
       'start_scan_to_end_scan', 'is_cutoff', 'cutoff_factor',
       'cutoff_timestamp', 'actual_distance_to_destination', 'osrm_time',
       'osrm_distance', 'factor', 'segment_actual_time', 'segment_osrm_time',
       'segment_osrm_distance', 'segment_factor', 'actual_time'],
      dtype='object')

In [22]:
df['segment_actual_time']

0         77.988686
1         68.517914
2         70.475386
3        101.876844
4         46.571301
            ...    
72153     55.856356
72154     35.976514
72155     44.101033
72156     37.771823
72157     54.154817
Name: segment_actual_time, Length: 72158, dtype: float64

In [5]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder

In [6]:
df[['trip_creation_date','trip_creation_clk']] = df['trip_creation_time'].str.split(' ', expand=True)
df[['trip_creation_year','trip_creation_month','trip_creation_day']] = df['trip_creation_date'].str.split('-', expand=True)
df[['trip_creation_hour','trip_creation_min','trip_creation_sec']] = df['trip_creation_clk'].str.split(':', expand=True)

df[['trip_creation_year','trip_creation_month','trip_creation_day','trip_creation_hour','trip_creation_min','trip_creation_sec']] = df[['trip_creation_year','trip_creation_month','trip_creation_day','trip_creation_hour','trip_creation_min','trip_creation_sec']].astype(float)

df = df.drop(['trip_creation_time','trip_creation_date','trip_creation_clk'], axis=1)

In [7]:
print(len(pd.unique(df['route_schedule_uuid'])))
df = df.drop(['route_schedule_uuid'], axis=1)

1470


In [8]:
print(len(pd.unique(df['route_type'])))

encoder = LabelBinarizer()
df['route_type'] = encoder.fit_transform(df['route_type'])

2


In [9]:
df['segment_key'] = df['trip_uuid'] + df['source_center'] + df['destination_center']
segment_cols = ['segment_actual_time', 'segment_osrm_distance', 'segment_osrm_time']

for col in segment_cols:
    df[col + '_sum'] = df.groupby('segment_key')[col].cumsum()
    
df = df.drop(['trip_uuid','source_center','source_name','destination_center','destination_name','segment_key'], axis=1)

In [10]:
df[['od_start_date','od_start_clk']] = df['od_start_time'].str.split(' ', expand=True)
df[['od_start_year','od_start_month','od_start_day']] = df['od_start_date'].str.split('-', expand=True)
df[['od_start_hour','od_start_min','od_start_sec']] = df['od_start_clk'].str.split(':', expand=True)

df[['od_start_year','od_start_month','od_start_day','od_start_hour','od_start_min','od_start_sec']] = df[['od_start_year','od_start_month','od_start_day','od_start_hour','od_start_min','od_start_sec']].astype(float)

df = df.drop(['od_start_time','od_start_date','od_start_clk'], axis=1)

In [11]:
df[['od_end_date','od_end_clk']] = df['od_end_time'].str.split(' ', expand=True)
df[['od_end_year','od_end_month','od_end_day']] = df['od_end_date'].str.split('-', expand=True)
df[['od_end_hour','od_end_min','od_end_sec']] = df['od_end_clk'].str.split(':', expand=True)

df[['od_end_year','od_end_month','od_end_day','od_end_hour','od_end_min','od_end_sec']] = df[['od_end_year','od_end_month','od_end_day','od_end_hour','od_end_min','od_end_sec']].astype(float)

df = df.drop(['od_end_time','od_end_date','od_end_clk'], axis=1)

In [12]:
print(len(pd.unique(df['is_cutoff'])))

encoder = LabelBinarizer()
df['is_cutoff'] = encoder.fit_transform(df['is_cutoff'])

2


In [13]:
df[['cutoff_timestamp_date','cutoff_timestamp_clk']] = df['cutoff_timestamp'].str.split(' ', expand=True)
df[['cutoff_timestamp_year','cutoff_timestamp_month','cutoff_timestamp_day']] = df['cutoff_timestamp_date'].str.split('-', expand=True)
df[['cutoff_timestamp_hour','cutoff_timestamp_min','cutoff_timestamp_sec']] = df['cutoff_timestamp_clk'].str.split(':', expand=True)

df[['cutoff_timestamp_year','cutoff_timestamp_month','cutoff_timestamp_day','cutoff_timestamp_hour','cutoff_timestamp_min','cutoff_timestamp_sec']] = df[['cutoff_timestamp_year','cutoff_timestamp_month','cutoff_timestamp_day','cutoff_timestamp_hour','cutoff_timestamp_min','cutoff_timestamp_sec']].astype(float)

df = df.drop(['cutoff_timestamp','cutoff_timestamp_date','cutoff_timestamp_clk'], axis=1)

In [23]:
print(df['cutoff_timestamp_hour'].isnull().sum())

KeyError: 'cutoff_timestamp_hour'

In [14]:
import numpy as np

In [15]:
x_df = df.drop(['UID','actual_time'], axis=1)
x = x_df.to_numpy()

y_df = df['actual_time']
y = y_df.to_numpy()

In [16]:
np.save('temp/x_train', x)
np.save('temp/y_train', y)

In [ ]:
x_df['segment_actual_time_sum'].head(25)